In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

batch_size = 8
num_classes = 10
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import torch
import torch.nn as nn

class DeepConvNet(nn.Module):
    def __init__(self, num_classes):
        super(DeepConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)

        # Dilated convolutional layers
        self.dilated_conv1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=2, dilation=2)
        self.dilated_conv2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=3, dilation=3)
        self.dilated_conv3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=4, dilation=4)
        self.dilated_conv4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=5, dilation=5)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.maxpool(x)
        x = self.relu(self.conv4(x))
        x = self.maxpool(x)
        x = self.relu(self.conv5(x))
        x = self.maxpool(x)
        
        x = self.relu(self.dilated_conv1(x))
        x = self.relu(self.dilated_conv2(x))
        x = self.relu(self.dilated_conv3(x))
        x = self.relu(self.dilated_conv4(x))
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder(root='/kaggle/input/autism-emotions/train/train', transform=transform)
test_dataset = ImageFolder(root='/kaggle/input/autism-emotions/test/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model = DeepConvNet(num_classes=len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


: 

In [ ]:
import torch
import torch.nn as nn

def summary(model, input_size):
    """
    Prints a summary of the PyTorch model.
    
    Args:
    - model: PyTorch model object
    - input_size: Tuple representing the input size (e.g., (3, 64, 64) for a 3-channel image with size 64x64)
    """
    device = next(model.parameters()).device
    print(f"{'Layer':<20} {'Input Shape':<30} {'Output Shape':<30} {'Param #':<15}")
    print("="*100)
    total_params = 0
    total_output = 0
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            input_shape = (1,) + input_size
            input_tensor = torch.rand(input_shape).to(device)
            output_tensor = layer(input_tensor)
            layer_params = sum(p.numel() for p in layer.parameters())
            total_params += layer_params
            total_output += output_tensor.numel()
            print(f"{str(layer):<20} {str(input_shape):<30} {str(output_tensor.shape):<30} {layer_params:<15}")
    print("="*100)
    print(f"Total params: {total_params}")
    print(f"Total output: {total_output}")

# summary(model, input_size=(3, 64, 64))


: 

In [ ]:
# Define a function to print model summary
def model_summary(model, input_size):
    print("Model Summary:")
    print("-" * 50)
    device = next(model.parameters()).device
    model = model.to(device)
    summary_str = str(model)
    num_params = sum(p.numel() for p in model.parameters())
    print(summary_str)
    print(f"Total parameters: {num_params}")
    print("-" * 50)

# Print model summary
model_summary(model, (3, 64, 64))


: 